In [1]:
# https://arxiv.org/html/2411.10541v1/
# https://arxiv.org/pdf/2411.10541

In [2]:
%pip install --upgrade langchain langchain-community langchain-core langchainhub langchain-qdrant langchain-ollama langchain-text-splitters langsmith

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
json_file_path = './products_v4.json'

In [4]:
import json
#read the content of json file
with open(json_file_path, 'r') as f:
    products_json = json.load(f)

In [5]:
for product in products_json[:5]:
  print(product)

{'Id': 1, 'Name': 'The Ordinary Peeling Solution', 'Brand': 'The Ordinary', 'Category': 'Exfoliating Peel', 'Price': '6,350.00', 'Ingredients': ['Glycolic Acid', 'Lactic Acid', 'Tartaric Acid', 'Citric Acid', 'Salicylic Acid', 'Sodium Hyaluronate Crosspolymer', 'Tasmannia Lanceolata Fruit/Leaf Extract'], 'Key Ingredients': ['Glycolic Acid', 'Salicylic Acid'], 'Benefits': ['Improves texture', 'clears pore congestion', 'targets uneven skin tone'], 'Potential Side Effects': ['Sun sensitivity', 'tingling', 'redness', 'potential irritation'], 'Natural': False, 'Concentrations': ['AHA 30%', 'BHA 2%'], 'Usage': 'Use once or twice a week on dry skin, Leave on for max 10 minutes and rinse, Avoid eye contour', 'Application Tips': 'Apply evenly using fingertips on clean/dry skin , Do not use on wet or compromised skin , Patch test recommended , Use sunscreen afterward', 'Skin Type': ['Normal', 'Oily', 'Combination'], 'Skin Concerns': ['Dullness', 'Uneven Texture', 'Enlarged Pores'], 'Average Rati

In [6]:
def display(value):
  if value is None or len(value) == 0:
    return 'Not specified'
  return value

def display_list(lst):
  if lst is None or len(lst) == 0:
    return 'Not specified'
  return ', '.join(lst)

In [7]:
def generate_product_profile(product):
  # Format the product profile
  product_profile = f"""
  # {product['Name']} - {product['Category']}
  ---

  ## Product Overview
  - Name: {display(product["Name"])}
  - Brand: {display(product["Brand"])}
  - Category: {display(product["Category"])}
  - Price: {display(product["Price"])} (in LKR)
  - Natural: {product["Natural"]}

  ## Ingredients
  - Key Ingredients: {display_list(product['Key Ingredients'])}
  - Concentrations: {display_list(product['Concentrations'])}
  - Full Ingredient List: {display_list(product['Ingredients'])}

  ## Benefits and Claims
  - Benefits: {display_list(product['Benefits'])}
  - Claims: {display_list(product['Claims'])}

  ## Usage and Application
  - Usage: {display(product['Usage'])}
  - Application Tips: {display(product['Application Tips'])}

  ## Skin Suitability
  - Suitable for Skin Types: {display_list(product['Skin Type'])}
  - Addresses Skin Concerns: {display_list(product['Skin Concerns'])}
  - For Sensitive Skin: {display(product['For Sensitive Skin'])}

  ## Safety Information
  - Potential Side Effects: {display_list(product['Potential Side Effects'])}
  - Allergens: {display_list(product['Allergens'])}

  ## Reviews and Ratings
  - Average Rating: {product['Average Rating']}/5
  - Customer Reviews:
  """
  # Append customer reviews with consistent indentation
  if product['Customer Reviews']:
    for i, review in enumerate(product['Customer Reviews']):
      if i == 0:
        product_profile += f"  - \"{review['review']}\" - {review['rating']} stars\n"
      else:
        product_profile += f"    - \"{review['review']}\" - {review['rating']} stars\n"
  else:
    product_profile += "    No customer reviews available.\n"

  # Append expert review
  product_profile += f"  - Expert Review: {display(product['Expert Review'])}"

  return product_profile

In [8]:
product_profile = generate_product_profile(products_json[0])
print(product_profile)


  # The Ordinary Peeling Solution - Exfoliating Peel
  ---

  ## Product Overview
  - Name: The Ordinary Peeling Solution
  - Brand: The Ordinary
  - Category: Exfoliating Peel
  - Price: 6,350.00 (in LKR)
  - Natural: False

  ## Ingredients
  - Key Ingredients: Glycolic Acid, Salicylic Acid
  - Concentrations: AHA 30%, BHA 2%
  - Full Ingredient List: Glycolic Acid, Lactic Acid, Tartaric Acid, Citric Acid, Salicylic Acid, Sodium Hyaluronate Crosspolymer, Tasmannia Lanceolata Fruit/Leaf Extract

  ## Benefits and Claims
  - Benefits: Improves texture, clears pore congestion, targets uneven skin tone
  - Claims: Clinically formulated, High-strength exfoliator

  ## Usage and Application
  - Usage: Use once or twice a week on dry skin, Leave on for max 10 minutes and rinse, Avoid eye contour
  - Application Tips: Apply evenly using fingertips on clean/dry skin , Do not use on wet or compromised skin , Patch test recommended , Use sunscreen afterward

  ## Skin Suitability
  - Suitable 

In [9]:
product_docs = []

for product in products_json:
  # Add formatted profile to the list
  product_docs.append({
    "template": generate_product_profile(product),
    "metadata": {
      "id": product["Id"],
      "name": product["Name"],
      "brand": product["Brand"],
      "category": product["Category"],
      "price": "LKR " + product["Price"]
    }
  })

In [10]:
len(product_docs)

100

In [11]:
product_docs[0]

{'template': '\n  # The Ordinary Peeling Solution - Exfoliating Peel\n  ---\n\n  ## Product Overview\n  - Name: The Ordinary Peeling Solution\n  - Brand: The Ordinary\n  - Category: Exfoliating Peel\n  - Price: 6,350.00 (in LKR)\n  - Natural: False\n\n  ## Ingredients\n  - Key Ingredients: Glycolic Acid, Salicylic Acid\n  - Concentrations: AHA 30%, BHA 2%\n  - Full Ingredient List: Glycolic Acid, Lactic Acid, Tartaric Acid, Citric Acid, Salicylic Acid, Sodium Hyaluronate Crosspolymer, Tasmannia Lanceolata Fruit/Leaf Extract\n\n  ## Benefits and Claims\n  - Benefits: Improves texture, clears pore congestion, targets uneven skin tone\n  - Claims: Clinically formulated, High-strength exfoliator\n\n  ## Usage and Application\n  - Usage: Use once or twice a week on dry skin, Leave on for max 10 minutes and rinse, Avoid eye contour\n  - Application Tips: Apply evenly using fingertips on clean/dry skin , Do not use on wet or compromised skin , Patch test recommended , Use sunscreen afterward\

In [12]:
print(product_docs[0]["template"])


  # The Ordinary Peeling Solution - Exfoliating Peel
  ---

  ## Product Overview
  - Name: The Ordinary Peeling Solution
  - Brand: The Ordinary
  - Category: Exfoliating Peel
  - Price: 6,350.00 (in LKR)
  - Natural: False

  ## Ingredients
  - Key Ingredients: Glycolic Acid, Salicylic Acid
  - Concentrations: AHA 30%, BHA 2%
  - Full Ingredient List: Glycolic Acid, Lactic Acid, Tartaric Acid, Citric Acid, Salicylic Acid, Sodium Hyaluronate Crosspolymer, Tasmannia Lanceolata Fruit/Leaf Extract

  ## Benefits and Claims
  - Benefits: Improves texture, clears pore congestion, targets uneven skin tone
  - Claims: Clinically formulated, High-strength exfoliator

  ## Usage and Application
  - Usage: Use once or twice a week on dry skin, Leave on for max 10 minutes and rinse, Avoid eye contour
  - Application Tips: Apply evenly using fingertips on clean/dry skin , Do not use on wet or compromised skin , Patch test recommended , Use sunscreen afterward

  ## Skin Suitability
  - Suitable 

In [13]:
from IPython.display import display, Markdown

display(Markdown(product_docs[0]["template"]))


  # The Ordinary Peeling Solution - Exfoliating Peel
  ---

  ## Product Overview
  - Name: The Ordinary Peeling Solution
  - Brand: The Ordinary
  - Category: Exfoliating Peel
  - Price: 6,350.00 (in LKR)
  - Natural: False

  ## Ingredients
  - Key Ingredients: Glycolic Acid, Salicylic Acid
  - Concentrations: AHA 30%, BHA 2%
  - Full Ingredient List: Glycolic Acid, Lactic Acid, Tartaric Acid, Citric Acid, Salicylic Acid, Sodium Hyaluronate Crosspolymer, Tasmannia Lanceolata Fruit/Leaf Extract

  ## Benefits and Claims
  - Benefits: Improves texture, clears pore congestion, targets uneven skin tone
  - Claims: Clinically formulated, High-strength exfoliator

  ## Usage and Application
  - Usage: Use once or twice a week on dry skin, Leave on for max 10 minutes and rinse, Avoid eye contour
  - Application Tips: Apply evenly using fingertips on clean/dry skin , Do not use on wet or compromised skin , Patch test recommended , Use sunscreen afterward

  ## Skin Suitability
  - Suitable for Skin Types: Normal, Oily, Combination
  - Addresses Skin Concerns: Dullness, Uneven Texture, Enlarged Pores
  - For Sensitive Skin: No

  ## Safety Information
  - Potential Side Effects: Sun sensitivity, tingling, redness, potential irritation
  - Allergens: Fragrances, Parabens

  ## Reviews and Ratings
  - Average Rating: 4.8/5
  - Customer Reviews:
    - "Skin feels baby-soft after one use!" - 5.0 stars
    - "Brightened my complexion overnight." - 4.5 stars
    - "Strong but very effective, definitely for experienced users." - 4.5 stars
  - Expert Review: Backed by clinical dermatology research

### Generate engaging product descriptions using product profiles

In [14]:
product_copywrite_prompt = """
You are an expert product copywriter. Based on the given product profile, write a professional and engaging product description that covers all the essential details, ingredients, benefits, usage instructions, and any special considerations.
Create a summary description of the skincare product based on the given structured product details.
Use only then given product information and phrase them as use see fit.
Don't remove any vital information from the product profile.
Be accurate and through.

Product Profile:
{product_profile}

Product Description:
"""

### Product Copywriter Agent

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_ollama import ChatOllama

def product_copywriter_agent(product_profile, prompt_template):
  prompt = PromptTemplate(input_variables=["product_profile"], template=prompt_template)

  agent = ChatOllama(
    base_url="http://216.81.248.11:11434",
    model="llama3.3:70b-instruct-q4_K_M",
    temperature=0,
  )

  product_copywrite_chain = LLMChain(llm=agent, prompt=prompt)

  product_description = product_copywrite_chain.run(product_profile=product_profile)

  return product_description

In [21]:
product_docs_1 = product_docs[:25]
product_docs_2 = product_docs[25:50]
product_docs_3 = product_docs[50:75]
product_docs_4 = product_docs[75:]

In [44]:
product_descriptions = []

for doc in product_docs_4:
  agent_response = product_copywriter_agent(doc["template"], product_copywrite_prompt)
  product_description = {
      "id": doc["metadata"]["id"],
      "metadata": doc["metadata"],
      "content": agent_response
  }
  product_descriptions.append(product_description)
  print(f"Product description for product {doc['metadata']['id']} generated")

Product description for product 76 generated
Product description for product 77 generated
Product description for product 78 generated
Product description for product 79 generated
Product description for product 80 generated
Product description for product 81 generated
Product description for product 82 generated
Product description for product 83 generated
Product description for product 84 generated
Product description for product 85 generated
Product description for product 86 generated
Product description for product 87 generated
Product description for product 88 generated
Product description for product 89 generated
Product description for product 90 generated
Product description for product 91 generated
Product description for product 92 generated
Product description for product 93 generated
Product description for product 94 generated
Product description for product 95 generated
Product description for product 96 generated
Product description for product 97 generated
Product de

In [45]:
len(product_descriptions)

25

In [46]:
output_file = "./product_descriptions.json"

In [48]:
def append_to_json(file_path, new_data):
    # Read existing data
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        data = []  # Initialize as list if file doesn't exist or is empty

    # Append new data
    data.append(new_data)

    # Write back to file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [49]:
append_to_json(output_file, product_descriptions)